## PRE-EXPERIMENT #2 - TENSORFLOW BERT CLASSIFICATION APPROACH FOR THE GOLF COURSE REVIEW DATA

In this file, we will be experimenting with the Tensorflow BERT model to create an end-to-end BERT 
classification model for the expanded golf course review dataset. In this experiment, we will be 
completing both the embedding creation and the classification model using BERT.

----